In [2]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import time
tf.random.set_seed(0)
np.random.seed(123)

In [35]:
def define_model():
    model = Sequential()
    model.add(Flatten(input_shape=(224,224,3)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(200, activation='relu'))

    # compile model
    model.add(Dense(1, activation='sigmoid'))
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [36]:
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    pyplot.savefig('vgg16_mlp_layers_plot.png')
    pyplot.close()

In [37]:
def run_test_harness():
    # define model
    model = define_model()
    print(model.summary())
    # create data generator
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_it = datagen.flow_from_directory('images/train/',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    test_it = datagen.flow_from_directory('images/test/',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    # fit model
    start_time = time.time()
    history = model.fit(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)
    end_time = time.time()
    # evaluate model
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
    training_time = end_time - start_time
    print("Training Time: %.2f seconds" % training_time)

    print("Training Loss:")
    print(history.history['loss'])
    print("Training Accuracy:")
    print(history.history['accuracy'])
    print("Testing Accuracy:")
    print('> %.3f' % (acc * 100.0))
    model.summary()
    summarize_diagnostics(history)

In [38]:
run_test_harness()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 150528)            0         
                                                                 
 dense_50 (Dense)            (None, 100)               15052900  
                                                                 
 dense_51 (Dense)            (None, 200)               20200     
                                                                 
 dense_52 (Dense)            (None, 200)               40200     
                                                                 
 dense_53 (Dense)            (None, 200)               40200     
                                                                 
 dense_54 (Dense)            (None, 1)                 201       
                                                                 
Total params: 15153701 (57.81 MB)
Trainable params: 15